In [1]:
from copy import deepcopy as copy
import pandas as pd
import numpy as np
import urllib.request
import re
import random

In [4]:
arquivo_txt = 'http://people.brunel.ac.uk/~mastjjb/jeb/orlib/files/mknapcb1.txt'

data = list(urllib.request.urlopen(arquivo_txt))

regex_numerica = re.compile('[0-9]+')

# Prepara problemas:

In [5]:
final_list = []
list_of_limited_weigth = []
weigth_limit = False
matrix_of_objects = None

for index, line in enumerate(data):
    try:
        
        # Tiramos apenas os numeros da linha
        splited_line = regex_numerica.findall(str(line))
        splited_line = [int(i) for i in splited_line]
        
        # Se for linha de peso, a proxima sera o inicio de um problema
        if weigth_limit:
            list_of_limited_weigth.append(splited_line)
            begining_of_problem = True
            weigth_limit = False
            continue
        
        # Iniciamos com a quantidade de problemas e a proxima é o sumario de um problema
        if index == 0 :
            number_of_problemns = splited_line[0]
            begining_of_problem= True
            continue

        # No inicio de um problema zeramos algumas variaveis
        if begining_of_problem:
            number_of_objects, number_of_slots, _ = splited_line
            begining_of_problem = False
            if matrix_of_objects is not None:
                final_list.append(copy(matrix_of_objects))
            matrix_of_objects = [[] for i in range(number_of_slots+1)]
            position = 0

        # Se nao é inicio do problema podemos adicionar os pesos a matrix de objetos
        else:    
            matrix_of_objects[position] += splited_line

        # Quando ja temos em um compartimento a quantidade de objetos podemos ir pro poximo slot
        # se esse slot for o ultimo, significa q a proxima linha é de pesos
        if len(matrix_of_objects[position]) == (number_of_objects):
            position+=1

            if position == (number_of_slots+1) : 
                weigth_limit = True

    except Exception as exp:
        print(exp)
final_list.append(copy(matrix_of_objects))

# Definindo o problema

In [6]:
# Escolha um problema de 0 a 29
INDEX_PROBLEMA = 5

In [7]:
problema = final_list[INDEX_PROBLEMA]
pesos = list_of_limited_weigth[INDEX_PROBLEMA]

beneficios = problema.pop(-1)
numero_de_compartimentos = len(problema)
numero_de_objetos = len(problema[0])
numero_de_compartimentos, numero_de_objetos

(5, 100)

# Algoritmo genético

In [8]:
import plotly.express as px
from copy import deepcopy as copy
from tqdm.notebook import tqdm
import time
import sys

MAX_GENERATIONS = 1000
CXPB = 0.65
MUTPB = 0.01
POPULATION_SIZE = 250

In [17]:
def inicia_populacao(numero_de_objetos, tamanho_populacao):
    individuo = [cria_individuo(numero_de_objetos) for i in range(tamanho_populacao)]
    return individuo

def cria_individuo(numero_de_objetos):
    # Cada individuo é composto por uma lista de tamanho 'numero_de_objetos'. Cada posição tem 0 ou 1
    # Se for carregar o objeto sua posicao possui valor 1 se nao possui valor 0
    
    individuo = [random.randint(0,1) for i in range(numero_de_objetos)]
    return individuo

def seleciona_populacao(pop_aumentada, number_of_pop, fits):
    # Selecao gulosa. Seleciona sempre os melhores
    lista_of_indices = sorted(range(len(fits)), key=lambda x: fits[x])[-number_of_pop:]
    return [i for index,i in enumerate(pop_aumentada) if index in lista_of_indices]

def cruzamento(filho1, filho2):
    ponto_de_corte = random.randint(1,len(filho1)-2)
    descendente1 = copy(filho1[:ponto_de_corte]) + copy(filho2[ponto_de_corte:])
    descendente2 = copy(filho2[:ponto_de_corte]) + copy(filho1[ponto_de_corte:])
    return descendente1, descendente2

def mutacao(individuo):
    novo_individuo = copy(individuo)
    for i in range(len(individuo)):
        if random.random() < MUTPB:
            novo_individuo[i] = 1 - novo_individuo[i]
    return novo_individuo

def calcula_fitness(individuo, problema, beneficios, pesos):
    beneficio = np.dot(individuo, beneficios)
    penalidade = 0
    for compartimento,peso_maximo in zip(problema, pesos):
        peso = np.dot(individuo, compartimento)
        extra_peso = peso-peso_maximo
        penalidade +=  extra_peso if extra_peso>0 else 0
    beneficio = beneficio - penalidade#(beneficio*0.001)
    return beneficio

250

In [20]:
g = 0
max_anterior = 0
contagem = 0
lista_metricas = []

pop = inicia_populacao(numero_de_objetos, POPULATION_SIZE)

# Inicia a evoluÃ§Ã£o
for g in tqdm(range(MAX_GENERATIONS)):

    # Aplica o cruzamento nos descendentes
    descendentes = []
    for filho1, filho2 in zip(pop[::2], pop[1::2]):

        # Se uma variÃ¡vel aleatÃ³ria for menor que a taxa de cruzamento
        if random.random() < CXPB:
            # Cruza dois filhos
            filho1, filho2 = cruzamento(filho1, filho2)
        descendentes += [filho1,filho2]

    # Aplica a mutaÃ§Ã£o aos descendentes
    for index,mutante in enumerate(descendentes):

        # Se uma variÃ¡vel aleatÃ³ria for menor que a taxa de mutaÃ§Ã£o
        descendentes[index] = mutacao(mutante)


    pop_aumentada = pop + descendentes
    # Salvamos todos os fitness para as estatÃ­sticas
    fits = [calcula_fitness(individuo, problema, beneficios, pesos) for individuo in pop_aumentada]

    # Selecionamos a pop
    pop = seleciona_populacao(pop_aumentada, len(pop), fits)

    try:
        fits = fits[:len(pop)]
        length = len(pop)
        mean = sum(fits) / length
        sum2 = sum(x * x for x in fits)
        std = abs(sum2 / length - mean ** 2) ** 0.5
        lista_metricas.append({
            'Média': mean,
            'Mínimo': min(fits),
            'Máximo': max(fits),
            'Desvio Padrão': std,
            'Geração': g,
        })
    except ZeroDivisionError:
        pass

    maximo = max(fits)

    if maximo == max_anterior:

        contagem += 1

    else:

        contagem = 0

    if (contagem == 50):
        break

    max_anterior = maximo
    
melhor_individuo = seleciona_populacao(pop, 1, fits)[0]
fitness_melhor_individuo = calcula_fitness(melhor_individuo, problema, beneficios, pesos)


print(f'Melhor individuo: {melhor_individuo}')
print(f'Fitness melhor individuo: {fitness_melhor_individuo}')
fig = px.line(pd.DataFrame(lista_metricas[1:]), x='Geração', y=['Média', 'Máximo'], title='Algorítmo genético')
display(fig)


Melhor individuo: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
Fitness melhor individuo: 15608
